In [24]:
import numpy as np
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader

from latent_geometry.model.mnist_vae import EncoderVAE, DecoderVAE
from latent_geometry.mapping import TorchModelMapping
from latent_geometry.visual.plotly import (
    create_topology_fig,
    create_topology_fig_geodesics,
)
from latent_geometry.manifold import LatentManifold
from latent_geometry.metric import EuclideanMetric

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from latent_geometry.config import NOTEBOOKS_DIR

ImportError: cannot import name 'NOTEBOOKS_DIR' from 'latent_geometry.config' (/home/quczer/school/mgr/latent-geometry/src/latent_geometry/config.py)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# initialize the model

encoder = EncoderVAE(init_channels=8, latent_dim=2, kernel_size=4, image_channels=1).to(
    device
)
decoder = DecoderVAE(init_channels=8, latent_dim=2, kernel_size=4, image_channels=1).to(
    device
)
params = [param for param in encoder.parameters()] + [
    param for param in decoder.parameters()
]

# set the learning parameters
lr = 0.001
epochs = 100
batch_size = 64
optimizer = optim.Adam(params, lr=lr)
criterion = nn.MSELoss(reduction="sum")

In [14]:
transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ]
)
# training set and train data loader
trainset = torchvision.datasets.MNIST(
    root=NOTEBOOKS_DIR / "input", train=True, download=True, transform=transform
)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
# validation set and validation data loader
testset = torchvision.datasets.MNIST(
    root=NOTEBOOKS_DIR / "input", train=False, download=True, transform=transform
)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

In [15]:
batch_size = 64

transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ]
)
testset = torchvision.datasets.MNIST(
    root=NOTEBOOKS_DIR / "input", train=False, download=True, transform=transform
)
testloader = DataLoader(testset, batch_size=500, shuffle=False)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = torch.load(NOTEBOOKS_DIR / "output" / "encoder", map_location=device)
decoder = torch.load(NOTEBOOKS_DIR / "output" / "decoder", map_location=device)

x = next(iter(trainloader))
z, mu, log_var = encoder(x[0])
reconstruction = decoder(z)

In [17]:
NUM_POINTS = 7000
WIDTH = 4
Z_SCALE = 5.0
SPLINE_POLY_DEG = 3
SIN_MULT = 1.5

In [18]:
# , manifold: LatentManifold, n_points: int = NUM_POINTS, width: float = WIDTH
def create_df(points, labels) -> pd.DataFrame:

    x_1 = points[:, 0]
    x_2 = points[:, 1]

    df = pd.DataFrame(np.stack([x_1, x_2, labels], axis=1), columns=["x", "y", "z"])

    # df = pd.DataFrame(data=np.array([x_1, x_2, labels]).transpose(), columns=["x", "y", "z"])

    # df['x']
    #
    return df


def create_background_df(
    manifold: LatentManifold, n_points: int = NUM_POINTS, width: float = WIDTH
) -> pd.DataFrame:

    x = np.random.rand(n_points, 1) * width * 2 - width
    y = np.random.rand(n_points, 1) * width * 2 - width

    df = pd.DataFrame(data=np.hstack([x, y]), columns=["x", "y"])
    df["cluster"] = (df.x > 0) + 2 * (df.y > 0)
    # df["z"] = np.sum(df.apply(lambda r: manifold.metric._mapping(r[:2], r[:2], r[:2]), axis=1))
    return df


def create_latent_fig(df: pd.DataFrame, three_d: bool = False) -> go.Figure:
    if three_d:
        return px.scatter_3d(df, x="x", y="y", z="z", color="z", opacity=1)
    else:
        return px.scatter(df, x="x", y="y", color="z", opacity=0.5)
        # return px.scatter(df, x="x", y="y", opacity=0.5)  # z tym działało

In [19]:
def create_topology_fig_geodesics_given_manifold(
    points,
    labels,
    centers_1: np.ndarray,
    centers_2: np.ndarray,
    mu_2: np.ndarray,
    log_var_2: np.ndarray,
    manifold: LatentManifold,
    num_lines: int,
    num_circles: int,
    line_length: float = 2.5,
    show_lines: bool = True,
    show_circles: bool = True,
) -> go.Figure:
    df_ = create_df(points, labels)
    background_trace = create_latent_fig(df_, three_d=False).data[0]

    return create_topology_fig_geodesics(
        centers_1,
        centers_2,
        mu_2,
        log_var_2,
        manifold,
        background_trace,
        num_lines,
        num_circles,
        line_length,
        show_lines,
        show_circles,
    )

In [20]:
def create_topology_fig_given_manifold(
    points,
    labels,
    centre: np.ndarray,
    manifold: LatentManifold,
    num_lines: int,
    num_circles: int,
    line_length: float = 10,
    show_lines: bool = True,
    show_circles: bool = True,
) -> go.Figure:
    # df_ = create_background_df(manifold)
    df_ = create_df(points, labels)
    background_trace = create_latent_fig(df_, three_d=False).data[0]

    return create_topology_fig(
        centre,
        manifold,
        background_trace,
        num_lines,
        num_circles,
        line_length,
        show_lines,
        show_circles,
    )

In [21]:
ambient_metric = EuclideanMetric(1024)
manifold_mnist = LatentManifold(
    TorchModelMapping(decoder, (2,), (1, 1, 32, 32)), ambient_metric
)

batch = next(iter(testloader))
points, mu, log_var = encoder(batch[0])

points = points.detach().numpy()
mu = mu.detach().numpy()
log_var = log_var.detach().numpy()
labels = batch[1]

# mu = mu[0]
# log_var = log_var[0]
# x = next(iter(testloader))
# points = encoder(x[0])[0]
# labels = x[1]
center = points[0]
mu = mu[0]
log_var = log_var[0]
centers_1 = points[:10]
centers_2 = points[10:20]

# create_topology_fig_given_manifold(
#     points=points,
#     labels=labels,
#     centre=center,
#     mu=mu,
#     log_var=log_var,
#     manifold=manifold_mnist,
#     num_lines=16,
#     num_circles=4,
# ).show()

In [22]:
create_topology_fig_given_manifold(
    points=points,
    labels=labels,
    centre=center,
    manifold=manifold_mnist,
    num_lines=16,
    num_circles=4,
).show()

KeyboardInterrupt: 

In [ ]:
ambient_metric = EuclideanMetric(1024)
manifold_mnist = LatentManifold(
    TorchModelMapping(decoder, (2,), (1, 1, 32, 32)), ambient_metric
)

batch = next(iter(testloader))
points, mu, log_var = encoder(batch[0])

points = points.detach().numpy()
mu = mu.detach().numpy()
log_var = log_var.detach().numpy()
labels = batch[1]

center = points[1]
mu = mu[1]
log_var = log_var[1]

create_topology_fig_given_manifold(
    points=points,
    labels=labels,
    centre=center,
    mu=mu,
    log_var=log_var,
    manifold=manifold_mnist,
    num_lines=16,
    num_circles=4,
).show()

TypeError: create_topology_fig() takes from 5 to 8 positional arguments but 10 were given